In [17]:
import os
import numpy as np
import fitz
import pandas as pd
from tqdm import tqdm
fol_path = r'\\isco-pipe.local\root\Common\EPC\Crowder Industrial\Crowder Santee Cooper Winyah ELG\Isometrics'

In [18]:
def extract_text_pymupdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

def do_extraction(full_pdf_path):
    # Extract text using PyMuPDF
    pymupdf_text = extract_text_pymupdf(full_pdf_path).split("\n")
    bill_of_materials = []
    start_collecting = False
    for line in pymupdf_text:
        if line=='N':
            break
        if line=='BILL OF MATERIALS':
            start_collecting = True
        bill_of_materials.append(line)
    return bill_of_materials

def make_table(bill_of_materials):
    new_table = []
    second_table = []
    unallowed = ['BILL OF MATERIALS','PIPE','FITTINGS','OLETS','FLANGES','VALVES','PIPE SUPPORTS']
    columns = ['ID','QTY','ND','DESCRIPTION','SUPPORT DETAIL']
    columns_seen = [0,0,0,0,0]
    table_to_input = new_table
    second_table_used = False
    for t_element in bill_of_materials:
        if not t_element.upper().strip() in unallowed:
            try:
                col_i = columns.index(t_element.upper().strip())
            except:
                col_i = -1
            if col_i!=-1:
                if columns_seen[col_i]==0:
                    if t_element=='SUPPORT DETAIL':
                        new_table = table_to_input
                        table_to_input = second_table
                        table_to_input.extend(columns[:-1])
                        second_table_used = True
                    columns_seen[col_i]=1
                elif columns_seen[col_i]==1:
                    continue
            table_to_input.append(t_element)
    if second_table_used:
        second_table = table_to_input
    else:
        new_table = table_to_input
    new_table = np.array(new_table).reshape(-1,4)
    if len(second_table)>0:
        second_table = np.array(second_table).reshape(-1,5)
    return new_table,second_table

def combine_tables(new_table,second_table,pdf_path):
    ndf = pd.DataFrame(new_table)
    ndf.columns = ndf.iloc[0]
    ndf = ndf.drop(0)
    if isinstance(second_table,np.ndarray):
        sdf = pd.DataFrame(second_table)
        sdf.columns = sdf.iloc[0]
        sdf = sdf.drop(0)
        combined = pd.concat([ndf, sdf])#.fillna(None)
        final = combined
    else:
        final = ndf
    final['FILE_PATH'] = os.path.split(pdf_path)[1]
    return final

def extract_bill_of_materials(pdf_path):
    bill_of_materials = do_extraction(pdf_path)
    t1,t2 = make_table(bill_of_materials)
    t = combine_tables(t1,t2,pdf_path)
    return t

def extract_bills_of_materials(fol_path):
    bills_of_materials = []
    errors = []
    for pdf in tqdm(os.listdir(fol_path)):
        pdf_path = os.path.join(fol_path,pdf)
        try:
            bills_of_materials.append(extract_bill_of_materials(pdf_path))
        except Exception as e:
            print((pdf_path,e))
            errors.append((pdf_path,e))
            
    return bills_of_materials,errors

In [20]:
bills_of_materials,errors = extract_bills_of_materials(fol_path)

 18%|██████████████▍                                                                  | 50/280 [00:46<03:21,  1.14it/s]

('\\\\isco-pipe.local\\root\\Common\\EPC\\Crowder Industrial\\Crowder Santee Cooper Winyah ELG\\Isometrics\\18-0-528-40603-PE01 - Isometric.pdf', ValueError('cannot reshape array of size 39 into shape (4)'))


 24%|███████████████████▋                                                             | 68/280 [01:02<03:01,  1.17it/s]

('\\\\isco-pipe.local\\root\\Common\\EPC\\Crowder Industrial\\Crowder Santee Cooper Winyah ELG\\Isometrics\\3-0-528-40401A-PE01 - Isometric.pdf', ValueError('cannot reshape array of size 39 into shape (4)'))


 25%|███████████████████▉                                                             | 69/280 [01:03<02:59,  1.18it/s]

...

('\\\\isco-pipe.local\\root\\Common\\EPC\\Crowder Industrial\\Crowder Santee Cooper Winyah ELG\\Isometrics\\6-0-528-40657-PE01 - Isometric.pdf', ValueError('cannot reshape array of size 14 into shape (5)'))


 87%|█████████████████████████████████████████████████████████████████████▋          | 244/280 [03:36<00:33,  1.09it/s]

('\\\\isco-pipe.local\\root\\Common\\EPC\\Crowder Industrial\\Crowder Santee Cooper Winyah ELG\\Isometrics\\8-0-528-40197-PE01 - Isometric.pdf', ValueError('cannot reshape array of size 19 into shape (5)'))


100%|████████████████████████████████████████████████████████████████████████████████| 280/280 [04:07<00:00,  1.13it/s]

('\\\\isco-pipe.local\\root\\Common\\EPC\\Crowder Industrial\\Crowder Santee Cooper Winyah ELG\\Isometrics\\8-0-528-40659-PE01 - Isometric.pdf', ValueError('cannot reshape array of size 14 into shape (5)'))


In [23]:
df = pd.concat(bills_of_materials)
len(df['FILE_PATH'].unique())

260

In [24]:
df.to_excel(os.path.join(fol_path,'extracted_boms.xlsx'))

In [25]:
with open(os.path.join(fol_path,'extracted_boms_errors.txt'),'w') as error_txt:
    for error in errors:
        error_txt.write(str(error)+'\n')